This notebook demonstrates the use of the One Time Pad.


0.   The first section generates the one time pad, and for convenience the corresponding encoding dictionaries.  This information is then saved as a json.
1.   The second section loads the json and converts a string into ciphertext.  The ciphertext is then saved as a textfile.
2.   The third section loads the json and text file, and converts the ciphertext back into the original string.

Variable names are kept unique by appending the section number.

In [1]:
import json
import numpy as np
rng = np.random.default_rng()

# One Time Pad

## Pad Parameters

In [2]:
# num chars for OTP
pad_size = 100
# preserve spaces in original message
incl_spaces = True

## Random Encoder (not as common, if used keep secret with one time pad)

In [3]:
alphabet = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O'
    , 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' '
]
if incl_spaces == False:
    del alphabet[-1]
keys = rng.choice(len(alphabet), size=len(alphabet) ,replace=False)
print(keys)

encoder0 = dict(zip(alphabet, keys))
decoder0 = {value: key for key, value in encoder0.items()}

[ 7 14  1  5  0 13 25  8 11  3 24 22  4 23  9 20 26 15 21 10 18  2 12 19
 16  6 17]


## Enumerated Encoder (more common)

In [4]:
encoder0 = {
    'A':0, 'B':1, 'C':2, 'D':3, 'E':4,'F':5, 'G':6, 'H':7, 'I':8, 'J':9, 'K':10,
    'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19,
    'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, ' ':26,
}
if incl_spaces == False:
    # del encoder[' ']
    print("Deleted space key from encoder")
decoder0 = {value: key for key, value in encoder0.items()}

## One Time Pad (keep secret)

In [5]:
otp_int0 = rng.choice(len(encoder0), size=pad_size ,replace=True)
otp_chars0 = [decoder0[i] for i in otp_int0]
otp_string0 = ''.join(otp_chars0)

In [6]:
display(str(encoder0))
display(otp_string0)

"{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, ' ': 26}"

'ERBFJBRIQRLYAABMEURGDVPQPVYQOOL EUUQAKHCQTSSNXFJTWUUIPQACBSYXDJXPGRVA QOVGWMQEBWWXRFJFCFSDOUEHKERDIZ'

In [7]:
otp_json = json.dumps({
    "OTP": otp_string0,
    "Encoder": str(encoder0),
    "Decoder": str(decoder0)
})
with open('OTP.json', 'w') as f:
    f.write(otp_json)

# The Message

In [8]:
with open('OTP.json', 'r') as f:
    otp_json1 = json.load(f)
otp_chars1 = otp_json1['OTP']
encoder1 = eval(otp_json1['Encoder'])
decoder1 = eval(otp_json1['Decoder'])

In [9]:
message_string1 = "The quick brown fox jumps over the lazy dog"
if incl_spaces == False:
    message_string1 = message_string1.upper().replace(' ', '')
else:
    message_string1 = message_string1.upper()
    # I want an explict False to trigger removal of spaces
message_int1 = [encoder1[i] for i in message_string1]
message_int1 = message_int1[0:pad_size]

In [10]:
message_string1

'THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG'

## Ciphertext to Send

In [11]:
otp_int1 = np.array([encoder1[i] for i in otp_chars1])

In [12]:
cipher_int1 = (otp_int1[0:len(message_int1)] + message_int1) % len(encoder1)
cipher_string1 = ''.join([decoder1[i] for i in cipher_int1])

In [13]:
display(cipher_string1)

'XYFEZVZK QMOOWOLJHNFMOAEGULKSEKSLYTAAIEBTGY'

In [14]:
with open('ciphertext.txt', 'w') as f:
    f.write(cipher_string1)

# Deciphering

In [15]:
with open('ciphertext.txt', 'r') as f:
    cipher_string2 = f.read()

In [16]:
with open('OTP.json', 'r') as f:
    otp_json2 = json.load(f)
otp_chars2 = otp_json2['OTP']
encoder2 = eval(otp_json2['Encoder'])
decoder2 = eval(otp_json2['Decoder'])

In [17]:
otp_int2 = np.array([encoder2[i] for i in otp_chars2])

In [18]:
cipher_int2 = [encoder2[i] for i in cipher_string2]
decipher_int2 = (cipher_int2 - otp_int2[0:len(cipher_string2)]) % len(encoder2)
decipher_string2 = ''.join([decoder2[i] for i in decipher_int2])
print(decipher_string2)

THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG


In [19]:
decipher_string2 == message_string1

True

In [20]:
all(cipher_int2 == cipher_int1)

True